In [2]:
ib.connect_db('ib://ewu/w4111-public/databases/w4111')

Connected to: ib://ewu/w4111-public/databases/w4111


## NOTE: Execute Nested Query (if an answer has multiple ones) to Output Result

### **Q1.1** Which store had the most sales in terms of total records in the table?

In [60]:
%%sql

select store, count(1)
from iowa
group by store
order by count(1) desc
limit 1;

,store,count
0,2633,8934


### Answer

store 2633

count 8934

### **Q1.2** At the store with the most total records (answer to Q1.1), what was the vendor number with most sale records? (The vendor that has the most records in the table?)

In [3]:
%%sql

select vendor_no, count(1)
from iowa
where store = 2633
group by vendor_no
order by count(1) desc
limit 1;


,vendor_no,count
0,260,1509


### Answer

vendor_no 260

count 1509

### **Q1.3**: For each zipcode, compute the single most purchased category_name by total `sale_bottles`.

a. We first need to get have a table with aggregated sale_bottles groupped by zipcode & category_name

In [62]:
%%sql
-- step 1
SELECT zipcode, category_name, sum(sale_bottles) sum_qty INTO TEMP zip_cate_sum
FROM iowa
GROUP BY zipcode, category_name;

{u'status': u'OK'}

b. With `zip_cate_sum` table, we can get the max `sum_qty` in a given zipcode with other groupby

In [63]:
%%sql
-- step 1
SELECT zipcode, category_name, sum(sale_bottles) sum_qty INTO TEMP zip_cate_sum
FROM iowa
GROUP BY zipcode, category_name;

-- step 2
SELECT zcs.zipcode, max(zcs.sum_qty) max_qty INTO TEMP zip_cate_sum_max
FROM zip_cate_sum zcs
GROUP BY zcs.zipcode;

{u'status': u'OK'}

c. use max bottle_qty sum and zipcode information, we can join above two tables to get category_name,
and use `ORDER BY` and `LIMIT` for expected output

In [64]:
%%sql
-- step 1
SELECT zipcode, category_name, sum(sale_bottles) sum_qty INTO TEMP zip_cate_sum
FROM iowa
GROUP BY zipcode, category_name;

-- step 2
SELECT zcs.zipcode, max(zcs.sum_qty) max_qty INTO TEMP zip_cate_sum_max
FROM zip_cate_sum zcs
GROUP BY zcs.zipcode;

-- step 3
SELECT c.zipcode, c.category_name, c.sum_qty
FROM zip_cate_sum c, zip_cate_sum_max b
WHERE b.zipcode = c.zipcode AND b.max_qty = c.sum_qty
ORDER BY c.sum_qty desc
LIMIT 5;

{u'status': u'OK'}

### Run Nested Query to Project Final Result on Q 1.3

In [65]:
%%sql
select c.zipcode, c.category_name, c.qty
from (
    select a.zipcode, max(a.qty) qty
    from (
        select zipcode, category_name, sum(sale_bottles) qty
        from iowa
        group by zipcode, category_name
    ) a
    group by a.zipcode
) b, 
(
    select zipcode, category_name, sum(sale_bottles) qty
    from iowa
    group by zipcode, category_name
) c
where b.zipcode = c.zipcode and b.qty = c.qty
order by c.qty desc
limit 5;


,zipcode,category_name,qty
0,52402,VODKA 80 PROOF ...,64368
1,52240,VODKA 80 PROOF ...,55597
2,50314,VODKA 80 PROOF ...,49893
3,50320,VODKA 80 PROOF ...,44763
4,51501,VODKA 80 PROOF ...,40812


### Answer

(zipcode, category_name, qty)

0   52402	VODKA 80 PROOF ...	64368

1	52240	VODKA 80 PROOF ...	55597

2	50314	VODKA 80 PROOF ...	49893

3	50320	VODKA 80 PROOF ...	44763

4	51501	VODKA 80 PROOF ...	40812

### **Q1.4**: This problem has two steps, you only need to return the value from second step.

__Compute the set of all liquors with the characters "tequila" (in lowercase) in its description (`im_desc`). The attribute `itemno` is the one that can serve as a unique identifier for a specific liquor.__

__Return the count of all distinct value of zipcode for all liquor stores that sold at least two of every tequila as defined in the previous sentence.__

a. Total count of # tequila can be made into one table

In [4]:
%%sql 
-- step 1
SELECT COUNT(DISTINCT(itemno)) INTO TEMP tequila_item_ctn
FROM iowa
WHERE im_desc LIKE '%tequila%';

{u'status': u'OK'}

b. with selection of tequila on description, and `group by` store and item, 
we can aggregate total sale_bottles, so that this `sum(sale_bottles)` is the __total sold bottles of one type of tequila at one store__

In [5]:
%%sql 
-- step 1
SELECT COUNT(DISTINCT(itemno)) item_cnt INTO TEMP tequila_item_ctn
FROM iowa
WHERE im_desc LIKE '%tequila%';

-- step 2
SELECT store, itemno, sum(sale_bottles) qty_sum INTO store_tequlia_bottle_sum
FROM iowa
WHERE im_desc LIKE '%tequila%'
GROUP BY store, itemno;

Error: ERROR: relation "store_tequlia_bottle_sum" already exists


c. We can select from `store_tequila_bottle_sum` table with qty_sum >= 2, to find out whether the sum bottle qty of a given type of tequila in a given is qualified.

After this SELECTION, we can apply a `GROUP BY` on store to get the total qualfied number of tequila of a given store.

In [6]:
%%sql 
-- step 1
SELECT COUNT(DISTINCT(itemno)) item_cnt INTO TEMP tequila_item_ctn
FROM iowa
WHERE im_desc LIKE '%tequila%';

-- step 2
SELECT store, itemno, sum(sale_bottles) qty_sum INTO TEMP store_tequlia_bottle_sum
FROM iowa
WHERE im_desc LIKE '%tequila%'
GROUP BY store, itemno;

-- step 3
SELECT stbs.store, count(*) n_qualified_tequila INTO TEMP store_qualified_tequila_item
FROM store_tequlia_bottle_sum stbs
WHERE stbs.qty_sum >= 2
GROUP BY stbs.store;

{u'status': u'OK'}

d. Join `store_qualified_tequila_item` and `tequila_item_ctn` to find out qualified store

In [71]:
%%sql 
-- step 1
SELECT COUNT(DISTINCT(itemno)) item_cnt INTO TEMP tequila_item_ctn
FROM iowa
WHERE im_desc LIKE '%tequila%';

-- step 2
SELECT store, itemno, sum(sale_bottles) qty_sum INTO TEMP store_tequlia_bottle_sum
FROM iowa
WHERE im_desc LIKE '%tequila%'
GROUP BY store, itemno;

-- step 3
SELECT stbs.store, count(*) n_qualified_tequila INTO TEMP store_qualified_tequila_item
FROM store_tequlia_bottle_sum stbs
WHERE stbs.qty_sum >= 2
GROUP BY stbs.store;

-- step 4
SELECT sqti.store INTO TEMP qualified_store
FROM store_qualified_tequila_item sqti, tequila_item_ctn tic
WHERE sqti.n_qualified_tequila = tic.item_cnt;

{u'status': u'OK'}

e. finally, join `qualified_store` on `iowa` to count distinct zipcode

In [72]:
%%sql 
-- step 1
SELECT COUNT(DISTINCT(itemno)) item_cnt INTO TEMP tequila_item_ctn
FROM iowa
WHERE im_desc LIKE '%tequila%';

-- step 2
SELECT store, itemno, sum(sale_bottles) qty_sum INTO TEMP store_tequlia_bottle_sum
FROM iowa
WHERE im_desc LIKE '%tequila%'
GROUP BY store, itemno;

-- step 3
SELECT stbs.store, count(*) n_qualified_tequila INTO TEMP store_qualified_tequila_item
FROM store_tequlia_bottle_sum stbs
WHERE stbs.qty_sum >= 2
GROUP BY stbs.store;

-- step 4
SELECT sqti.store INTO TEMP qualified_store
FROM store_qualified_tequila_item sqti, tequila_item_ctn tic
WHERE sqti.n_qualified_tequila = tic.item_cnt;

-- step 5
SELECT count(distinct(i.zipcode))
FROM qualified_store qs, iowa i
WHERE qs.store = i.store;

{u'status': u'OK'}

### Run Nested Query to Project Final Result on Q 1.4

In [7]:
%%sql
-- zip of all stores that have sold at least 2 of each tequila, having tequila in description

select count(distinct(e.zipcode))
from 
(
    select b.store, b.ctn
    from (
        select a.store, count(1) ctn
        from (
            select store, itemno, sum(sale_bottles) ctn
            from iowa
            where im_desc like '%tequila%'
            group by store, itemno
        ) a
        where a.ctn >= 2
        group by a.store
    ) b, 
    (
        select count(distinct(itemno)) ctn
        from iowa
        where im_desc like '%tequila%'
    ) c
    where b.ctn = c.ctn
) d
join iowa e
on e.store = d.store;

,count
0,110


### Answer:

count: 110

### **Q1.5:** What is the item attribute of the liquor with the highest state bottle cost, (as defined below), across all of iowa?

a. `GROUP BY` store and item to compute average state_bottle_retail per bottle_volume_ml for an item within store

In [79]:
%%sql
-- step 1
SELECT store, itemno, avg(state_bottle_retail/bottle_volume_ml) as cost INTO TEMP storewise_cost
FROM iowa
WHERE bottle_volume_ml > 0
GROUP BY store, itemno;

{u'status': u'OK'}

b. average across store for final result

In [80]:
%%sql
-- step 1
SELECT store, itemno, avg(state_bottle_retail/bottle_volume_ml) as cost INTO TEMP storewise_cost
FROM iowa
WHERE bottle_volume_ml > 0
GROUP BY store, itemno;

-- step 2
SELECT s.itemno, avg(s.cost) overall_cost
FROM storewise_cost s
GROUP BY itemno
ORDER BY avg(s.cost) DESC
LIMIT 1;

{u'status': u'OK'}

### Run Nested Query to Project Final Result on Q 1.5

In [82]:
%%sql
-- has highest state btl cost

select itemno, avg(per_store.cost) overall_cost
from (
    select store, itemno, avg(state_bottle_retail/bottle_volume_ml) as cost
    from iowa
    where bottle_volume_ml > 0
    group by store, itemno
) per_store
group by itemno
order by avg(per_store.cost) desc
limit 1;

,itemno,overall_cost
0,927368,11.5999999999999996


### Answer

itemno: 927368

### Q1.6 (Answer not Limited to those)
`Python Pros:`

Python can perform more complicated logic on data, for example we can implement algorithms in an imperative 
programming language such as Python but not in SQL.

With python, we can achieve complex data visualizations over web applications, which cannot be done through SQL.

`Python Cons:`

Python syntax on data manipulate is obscure sometimes.

Python data manipulate is hard to recover and rollback.


`SQL Pros:`

SQL is specialized to data transformations.

SQL is safer and more efficient because DBMS knows how to analyze it.

SQL provides ACID properties for transactions in database.

`SQL Cons`:

Require cross-language data pipeline for brining SQL analysis into product.